# [데이터 로딩, 저장, 파일 형식]

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
%pwd

'F:\\Jupyternotebook\\version_3.0\\pydataAnalysis'

## 6.1 텍스트 파일 이용방법
- read_csv : url 또는 파일과 유사한 객체(.csv)로부터 구분된 데이터를 읽어옴(,: 쉼표)
- read_table : 구분자('\t')을 기본으로 하는 파일을 읽어옴
- read_fwf : 고정폭 컬럼 형식에서 데이터를 읽어옴(구분자 없는 데이터)
- read_clipboard : 웹페이지에서 표를 긁어올때 쓰는 함수.

#### DataFrame을 읽어오는 함수로 몇가지 옵션
- 색인 : 반환하는 dataframe에서 하나 이상의 색인 지정 가능(아무것도 지정안할수도 있음)
- 자료형 추론과 데이터 변환
- 날짜 분석 : 여러컬럼에 걸쳐있는 날짜와 시간정보를 하나의 컬럼에 조합해서 결과반영.
- 반복 : 여러 파일에 걸쳐있는 자료를 반복적으로 읽어올 수 있음.
- 정제되지 않은 데이터 처리
- (unix : cat 명령어, windows : type 명령어)

In [7]:
df = pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_table('ch06/ex1.csv',sep=',') # seq : 구분자!

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
pd.read_csv('ch06/ex2.csv', header=None) # 컬럼이름 자동설정(숫자)
# 컬럼이름 지정.
pd.read_csv('ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [18]:
names = ['a', 'b', 'c', 'd', 'message']
# index_col : 색인으로 지정할 컬럼 이름 설정.
pd.read_csv('ch06/ex2.csv', names=names, index_col='message') 

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [23]:
# 계층적 색인 지정 >> index_col=['key1', 'key2'] : 컬럼번호나 이름을 리스트로 남김.
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [25]:
# 고정된 구분자 없이 공백이나 다른 패턴으로 필드를 구분시
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [31]:
# read_table에 정규표현식 : \s+(여러개의 공백문자로 필드가 구분되어 있으므로...)
result = pd.read_table('ch06/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [34]:
list(open('ch06/ex4.csv'))

['# hey!\n',
 'a,b,c,d,message\n',
 '# just wanted to make things more difficult for you\n',
 '# who reads CSV files with computers, anyway?\n',
 '1,2,3,4,hello\n',
 '5,6,7,8,world\n',
 '9,10,11,12,foo']

In [35]:
# skiprows : 스킵되는 로우의 위치.(원래 문장상)
pd.read_csv('ch06/ex4.csv', skiprows=[0, 2, 3]) # 첫번째, 세번째, 네번째 제외.

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [37]:
# 누락값(NA,-1,#IND,NULL 등) 처리 
result = pd.read_csv('ch06/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [40]:
pd.isnull(result) # 누락값(nan) 확인 불리언.

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [45]:
# na_values 옵션 : 리스트나 문자열 집합을 받아서 누락값을 처리.
result = pd.read_csv('ch06/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [47]:
# 열마다 다른 NA문자를 사전값으로 넘겨 처리.
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


[read_csv/read_table 함수 인자]
- delimiter or seq : 필드를 구분하기 위해 사용할 연속된 문자나 정규표현식
- header : 컬럼의 이름으로 사용할 로우의 번호 , 기본값은 0, 헤더가 없으면 None
- index_col : 색인으로 사용할 컬럼번호나 이름, 계층적 색인을 지정할 경우 리스트를 넘김
- na_values : NA값으로 처리할 값들의 나열
- parse_dates : 날짜를 datatime으로 변환할지의 여부, 기본값은 False, True일 경우 모든 컬럼에 다 적용.
- converters : 변환시 컬럼에 적용할 함수 지정 (예: {'foo' : f} 'foo'컬럼에 f함수 적용.
- nrows : 파일의 첫 일부만 읽어올때 처음 몇 줄을 정하는 수.
- iterator : 파일을 조금씩 읽을때 사용하도록 TextParser객체를 반환.

### 6.1.1  텍스트 파일 조금씩 읽기.

In [49]:
result = pd.read_csv('ch06/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [51]:
pd.read_csv('ch06/ex6.csv', nrows=5) # nrow : 읽어올 행 개수

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [53]:
# chunksize : 여러조각에 나누어서 읽고 싶을때의 옵션으로 로우의 개수
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)
chunker

In [59]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.order(ascending=False)

AttributeError: 'Series' object has no attribute 'order'

In [62]:
tot[:10]

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
5    157.0
6    166.0
7    164.0
8    162.0
9    150.0
dtype: float64

### 6.1.2 : 데이터를 텍스트 형식으로 기록

In [64]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [67]:
data.to_csv('ch06/out.csv')
list(open('ch06/out.csv'))

[',something,a,b,c,d,message\n',
 '0,one,1,2,3.0,4,\n',
 '1,two,5,6,,8,world\n',
 '2,three,9,10,11.0,12,foo\n']

In [69]:
data.to_csv(sys.stdout, sep='|') # seq 구분자 변경

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [71]:
data.to_csv(sys.stdout, na_rep='NULL') # na_rep : NULL값으로 변경

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [73]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [76]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [78]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')

In [80]:
Series.from_csv('ch06/tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 6.1.3 : 수동으로 구분형식 처리하기.

In [82]:
import csv
f = open('ch06/ex7.csv')

reader = csv.reader(f)

In [86]:
for line in reader: # "를 제외한 튜플을 얻을수 있음.
    print(line)

In [88]:
lines = list(csv.reader(open('ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [92]:
#  csv.Dialect 를 상속받아 새로운 클래스 정의.
class my_dialect(csv.Dialect):
    lineterminator = '\n' # 파일 저장시 사용할 개행문자
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [94]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [97]:
f = open('mydata.csv')
reader = csv.reader(f)
for line2 in reader:
    print(line2)

['one;two;three']
['1;2;3']
['4;5;6']
['7;8;9']


### 6.1.4: JSON 데이터
- 웹브라우저와 다른 애플리케이션이 HTTP요청으로 데이터를 보낼때 널리 사용하는 표준파일형식 중 하나

In [98]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [100]:
import json
result = json.loads(obj)  # json을 불러옴.
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [102]:
asjson = json.dumps(result) # 파이썬 객체를 json으로 변환

In [3]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

NameError: name 'result' is not defined

### 6.1.5: XML과 HTML: 웹 내용 긁어오기.

In [8]:
import requests
from lxml.html import parse # 아주 큰 파일을 빠르게 처리할 수 있는 라이브러리.
# (http://lxml.de)
from io import StringIO
# (예시: 야후! 금융에서 서비스하고 있는 주신 선물 옵션 데이터)
text = requests.get('http://finance.yahoo.com/q/op?s=AAPL+Options').text
parsed = parse(StringIO(text))

doc = parsed.getroot() # << HTML 태크가 추출되어있음 : table 태그를 중점!

In [10]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x268a7946e08>,
 <Element a at 0x268a7946e58>,
 <Element a at 0x268a7946ea8>,
 <Element a at 0x268a7946ef8>,
 <Element a at 0x268a7946f48>]

In [16]:
lnk = links[28]
lnk
lnk.get('href') 
lnk.text_content() # 링크 이름을 가져오려는 메서드

'S&P 500'

In [18]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['https://smallbusiness.yahoo.com',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US&id=SLN2310',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US',
 'https://yahoo.uservoice.com/forums/382977',
 'http://info.yahoo.com/privacy/us/yahoo/',
 'http://info.yahoo.com/relevantads/',
 'http://info.yahoo.com/legal/us/yahoo/utos/utos-173.html',
 'https://twitter.com/YahooFinance',
 'https://facebook.com/yahoofinance',
 'http://yahoofinance.tumblr.com']

In [22]:
# table 태그를 찾아 가져옴
tables = doc.findall('.//table')
calls = tables[1]
puts = tables[2]

In [24]:
rows = calls.findall('.//tr') # tr 태그 내용

In [26]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [29]:
_unpack(rows[0], kind='th') # th태의 헤더내용 가져오기.

['Contract Name',
 'Last Trade Date',
 'Strike',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [33]:
_unpack(rows[1], kind='td') # td안의 데이터 가져오기

['AAPL170901C00095000',
 '2017-07-21 11:52PM EDT',
 '95.00',
 '54.89',
 '55.05',
 '55.65',
 '0.00',
 '-',
 '1',
 '1',
 '0.00%']

In [36]:
# 긁어온 데이터를 dataframe으로 변환.
# 변환시 숫자 데이터지만 문자열로 변환되어 있어 수동으로 처리해야함.
# pandas는 TextParser 클래스가 자동 형 변환을 수행

from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

In [38]:
call_data = parse_options_data(calls)
put_data = parse_options_data(puts)
call_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL170901C00095000,2017-07-21 11:52PM EDT,95.0,54.89,55.05,55.65,0.00,-,1,1,0.00%
1,AAPL170901C00105000,2017-08-25 11:49PM EDT,105.0,51.26,54.60,55.20,0.00,-,1,1,129.69%
2,AAPL170901C00110000,2017-08-24 12:15PM EDT,110.0,49.05,49.60,50.20,0.00,-,10,66,116.41%
3,AAPL170901C00115000,2017-08-25 10:15AM EDT,115.0,45.35,44.80,45.05,1.20,+2.72%,14,143,110.55%
4,AAPL170901C00120000,2017-08-09 2:29PM EDT,120.0,40.68,37.15,38.05,0.00,-,20,20,0.00%
5,AAPL170901C00125000,2017-08-02 9:57AM EDT,125.0,32.80,30.10,31.00,0.00,-,1,1,0.00%
6,AAPL170901C00130000,2017-08-21 3:10PM EDT,130.0,26.65,29.65,30.15,0.00,-,20,59,68.75%
7,AAPL170901C00131000,2017-08-21 3:10PM EDT,131.0,25.65,28.65,29.20,0.00,-,120,130,70.70%
8,AAPL170901C00132000,2017-08-24 11:24AM EDT,132.0,27.25,27.60,28.15,0.00,-,5,20,57.03%
9,AAPL170901C00133000,2017-08-25 10:19AM EDT,133.0,27.50,26.60,27.10,1.95,+7.63%,1,20,81.45%


#### lxml.objectify 이용해 XML 파싱하기.

In [45]:
from lxml import objectify
path = './ch06/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot() # root 노드에 대한 참조를 얻어옴.

In [47]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [49]:
perf = DataFrame(data)
perf # INDICATOR 의 태그 안의 태그가 헤더(컬럼이름.)

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [56]:
root

<Element PERFORMANCE at 0x268a7943948>

In [57]:
root.get('href')

In [59]:
root.text

## 6.2 이진 데이터 형식
- pickle : 데이터를 효율적으로 쉽게 저장하는 방법으로 파이썬의 기본 내장으로 있는 직렬화 이진형식 저장.(pandas는 pickle을 이용하여 데이터를 저장하는 save가 있음)

In [64]:
frame = pd.read_csv('ch06/ex1.csv')
frame 
frame.to_pickle('ch06/frame_pickle')
# pickle로 저장
# 오래 보관할 필요가 없는 데이터에만 추천.(오랜시간동안 데이터를 저장할 거라도 보장이 힘듬.)

In [66]:
pd.read_pickle('ch06/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.2.1: HDF5 형식 사용하기.
- HDF5 : Hierarchical Data Format
- 이진형식으로 저장된 대용량의 과학자료를 효율적으로 읽고 쓸 수 있는 다양한 라이브러리중 하나.
- 자바, 파이썬, Matlab언어를 위한 인터페이스를 제공하는 C라이브러리.
- 온더플라이(on-the-fly, 실시간) 압축을 지원 반복되는 패턴을 가진 데이터를 좀 더 효과적으로 저장가능.
- 파이썬 라이브러리 : PyTables, h5py

In [70]:
# HDFStore : pandas의 PyTables의 가벼운 사전클래스
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [72]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [74]:
store.close()
# os.remove('mydata.h5')

### 6.2.2: 마이크로소프트 엑셀파일에서 데이터 읽기
- ExcelFile 클래스 >> 엑셀 2003 이후 버전의 데이터를 읽어올수 있음
- 이 클래스는 xlrd, openpyxl 패키지 이용(두 패키지 설치필요)

## 6.3 HTML, 웹 API와 함께 사용하기

In [76]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

<Response [200]>

In [78]:
data[:5]

[{'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'FREQUENCY': 'M',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'INDICATOR_UNIT': '%',
  'MONTHLY_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'PERIOD_MONTH': 1,
  'PERIOD_YEAR': 2008,
  'YTD_ACTUAL': 96.9,
  'YTD_TARGET': 95.0},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\

In [80]:
issue_labels = DataFrame(data)
issue_labels

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


## 6.4 데이터베이스와 함께 사용하기.

In [82]:
import sqlite3 # 데이터베이스 

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [84]:
# 데이터 입력
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [85]:
cursor = con.execute('select * from test') # select 쿼리 수행시 튜플 리스트 반환.
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [87]:
cursor.description # 속성.

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [90]:
DataFrame(rows, columns=list(zip(*cursor.description))[0]) # 속성지정.

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


In [91]:
# pandas에서 sql의 read_sql로 간단히 처리..!
import pandas.io.sql as sql 
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
